## Calculate correlations between two datasets

#Adapted from code written by Allegra Aron available at: https://github.com/allegra-aron/Stromatolite_analysis/blob/main/correlations_between_two_feature_tables.ipynb

**Input file**: a .csv file in which the relative abundances of MS features (dataset 1, ds1) and microbial families (dataset 2, ds2) are listed consecutively. Each column represents a push core sample (see Table S0 for site key). Each row represents a data point (MS feature or microbial taxon), for which the ID is given in the first column.

**Parameters**:

In [ ]:
input_file <- "peptidemz_microbefamily_correlation_input.csv"
rows_ds1 <- 1:307 # which rows (not including header) contain contain peptidemz abundances (dataset 1)?
rows_ds2 <- 308:1093 # which rows (not including header) contain microbe family abundances (dataset 2)?
prev_filt <- 0.1 # minimum % of samples that a feature must be observed in
norm_ds1 <- TRUE # do you want to normalize the first dataset?
norm_ds2 <- FALSE # do you want to normalize the second dataset?
scale_features <- TRUE # do you want to scale prior to correlation?
padj_meth <- "BH" # method to adjust for multiple hypothesis tests (can change to "BH" to be less stringent)
p_thresh <- 0.05 # alpha for adjusted pvalues

## Load libraries

In [ ]:
install.packages("Hmisc")

In [ ]:
install.packages("corrplot")

In [ ]:
#install.packages('d3heatmap')

In [ ]:
install.packages("devtools")

In [ ]:
if (!require("devtools")) install.packages("devtools")
devtools::install_github("rstudio/d3heatmap")

In [ ]:
install.packages("pheatmap")

In [ ]:
install.packages("reshape2")

In [ ]:
library(d3heatmap)
library(Hmisc)
library(htmlwidgets)
library(pheatmap)
library(reshape2)
library(RColorBrewer)

In [ ]:
#?d3heatmap

## Read in data

In [ ]:
data <- read.csv('peptidemz_microbefamily_correlation_input.csv')

In [ ]:
Check that the row numbers provided are correct

In [ ]:
if (nrow(data) == length(c(rows_ds1, rows_ds2))) {message("Looks good!")} else message("Check row numbers again")
message(nrow(data))

## Format data

In [ ]:
datat <- t(data) # transpose so that samples are in rows and features are in columns
colnames(datat)<- datat[1,] # feature names are now the first row, make this row the column names
datat <- datat[-1,] # then remove it
datat <- as.data.frame(datat,stringsAsFactors=F)
datat <- as.data.frame(sapply(datat, as.numeric)) # make all values numeric
rownames(datat) <- colnames(data)[-1] # use sample names as row names

## Separate the two datasets for filtering/normalization

In [ ]:
ds1 <- datat[,rows_ds1]
ds2 <- datat[,rows_ds2]

## Prevalence filter

Since there are so many features, we can reduce the number based on how many samples have the feature. For example, if the cutoff is 10%, we only keep features observed in at least 10% of samples

In [ ]:
ds1_filt <- ds1[,apply(ds1, 2, function(x) {sum(x > 0) > prev_filt*nrow(ds1)})]
ds2_filt <- ds2[,apply(ds2, 2, function(x) {sum(x > 0) > prev_filt*nrow(ds2)})]

## Normalize data by total sum scaling

In the original code, they added 1 to every feature but I'm not sure why?

In [ ]:
if (norm_ds1) {
  ds1_norm <- t(apply(ds1_filt, 1, function(x) {x/sum(x)})) # for each sample (row) divide each feature by the sum of all features
} else ds1_norm <- ds1_filt

if (norm_ds2) {
  ds2_norm <- t(apply(ds2_filt, 1, function(x) {x/sum(x)})) 
} else ds2_norm <- ds2_filt

## Scale data

In [ ]:
if (scale_features) {
  ds1_norm <- scale(ds1_norm)
  ds2_norm <- scale(ds2_norm)
}

## Calculate correlations between ds1 (x) and ds2 (y)

In [ ]:
cor_mat <- Hmisc::rcorr(x = as(ds1_norm, "matrix"), y = as(ds2_norm, "matrix"), type = "spearman") # spearman recommended for microbiome data

# get correlations
cor_r <- cor_mat$r[1:ncol(ds1_norm), -c(1:ncol(ds1_norm))] # removing ds1~ds1 and ds2~ds2 correlations, so we only keep ds1~ds2 correlations
# get pvalues
cor_p <- cor_mat$P[1:ncol(ds1_norm), -c(1:ncol(ds1_norm))]

In [ ]:
head(cor_mat)
head(cor_r)
head(cor_p)

## Heatmap

In [ ]:
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
 
# install and load package

BiocManager::install("DESeq2")
library("DESeq2")

In [ ]:
pheatmap(t(cor_r),  clustering_method="ward.D", clustering_distance_cols="canberra",
         show_colnames = TRUE,show_rownames = TRUE, cluster_rows = TRUE, cluster_cols = TRUE, fontsize = 1, 
         filename = "corr_coef_all_peptidemz_family_BH_.pdf")

## Interactive heatmap

(this file was too big to open on my laptop!)

In [ ]:
map <- d3heatmap(t(cor_r), distfun=function(x) dist(x, method="canberra"), 
                 hclustfun=function(x) hclust(x, method="ward.D"),
                 col = colorRampPalette(rev(brewer.pal(n = 7, name = "RdBu")))(100))
saveWidget(map, "corr_coef_all_peptidemz_family_BH_noyellow.html")

## Remove correlations with padj < alpha

In [ ]:
#install.packages("reshape")
#library(reshape)
install.packages("reshape2")
library(reshape2)

In [ ]:
hold_r <- melt(cor_r) # putting all values in one column
hold_p <- melt(cor_p)
hold_p$padj <- p.adjust(hold_p$value, method = padj_meth) # adjust pvals for multiple hypothesis tests

hold_r_sig <- hold_r[hold_p$padj < p_thresh,]
hold_p_sig <- hold_p[hold_p$padj < p_thresh,]

# new matrix with only significant correlations (insignificant correlations are set to 0)
corr_r_sig <- dcast(hold_r_sig, Var1~Var2, fill = 0)
row.names(corr_r_sig) <- corr_r_sig$Var1 # re-assign row names as feature names then remove that column
corr_r_sig <- corr_r_sig[,-1]

## Plot heatmap again

In [ ]:
map <- d3heatmap(t(corr_r_sig), distfun=function(x) dist(x, method="canberra"), 
                 hclustfun=function(x) hclust(x, method="ward.D"),
                 col = colorRampPalette(rev(brewer.pal(n = 7, name = "RdBu")))(100))
saveWidget(map, "corr_coef_sig_peptidemz_family_BH.html")

In [ ]:
pheatmap(t(corr_r_sig),  clustering_method="ward.D", clustering_distance_cols="canberra",
         show_colnames = TRUE,show_rownames = TRUE, cluster_rows = TRUE, cluster_cols = TRUE, fontsize = 1,color = colorRampPalette((brewer.pal(n = 7, name =
  "RdBu")))(100), 
         filename = "corr_coef_sig_peptidemz_family_BH.pdf")

## Save tables

In [ ]:
colnames(hold_r)[3] <- "corr_coef" # fix column names
colnames(hold_p)[3] <- "pval"
to_save <- cbind(hold_r, hold_p[,3:4]) # join coefs and pvals
write.csv(to_save, "corr_coef_all_peptidemz_family_BH.csv", row.names = FALSE) # save table

# save only correlations with significant pval
write.csv(subset(to_save, to_save$pval < p_thresh), "corr_coef_sig_peptidemz_family_BH.csv", row.names = FALSE) 

In [ ]:
hist(-log10(to_save$pval),breaks=100)

In [ ]:
hist(to_save$corr_coef,breaks=100)
dev.copy(png,'histogram_sig_peptidemz_family_BH.png')
dev.off()

In [ ]:
hist(to_save$corr_coef,-1:1, breaks=100)

In [ ]:
h_1 <- hist(to_save$corr_coef,-1:1, breaks=100)

In [ ]:
h_1$breaks

In [ ]:
h_1$counts